In [92]:
import pandas as pd
import numpy as np
import stat
import re
import warnings
warnings.filterwarnings("ignore")

In [93]:
#cd 'cleaning'

In [94]:
#analysis = pd.read_csv("FF_HEALTH_DC3_ANALYSIS.csv")
roster = pd.read_csv("../HUTKA_ROSTER_V2.CSV")
dictionary = pd.read_csv("../HUTKA_CHECKIN01_QUESTION_DICTIONARY.csv")
raw = pd.read_csv("HUTKA_CHECKIN02_RAW.csv")

full_cleaned_data = pd.read_csv("full_cleaned.csv")

In [95]:
quant = [str(x) for x in list(dictionary[dictionary["type"] == 'quantitative']["question_id"])]

numerical_data = full_cleaned_data[['TeamNumber', 'TeammateNumber','2.1',
 '2.2',
 '2.3',
 '4.1',
 '4.2',
 '4.3',
 '4.4', 
 '4.5',
 '4.6',
 '4.7',
 '5.1',
 '5.2',
 '5.3',
 '5.4', 
 '5.5',
 #'5.6',
 #'5.7',
 '16.1', '16.2', '16.3', '16.4', '16.5',# '16.6', '16.7',
'17.1', '17.2', '17.3', '17.4', '17.5', #'17.6', '17.7',
'18.1', '18.2', '18.3', '18.4', '18.5', #'18.6', '18.7',
'19.1', '19.2', '19.3', '19.4', '19.5', #'19.6', '19.7'
                                   ]]
numerical_data = numerical_data.replace({"No Response" : np.nan})
numerical_data.head()

,TeamNumber,TeammateNumber,2.1,2.2,2.3,4.1,4.2,4.3,4.4,4.5,...,18.1,18.2,18.3,18.4,18.5,19.1,19.2,19.3,19.4,19.5
0,1.0,1.0,5.0,6.0,5.0,5.0,5.0,6.0,6.0,6.0,...,5.0,6.0,6.0,6.0,5.0,5.0,6.0,5.0,5.0,4.0
1,1.0,2.0,7.0,6.0,7.0,7.0,7.0,6.0,7.0,6.0,...,7.0,7.0,7.0,7.0,6.0,7.0,7.0,7.0,7.0,7.0
2,1.0,3.0,5.0,6.0,6.0,6.0,5.0,7.0,6.0,6.0,...,6.0,6.0,6.0,6.0,2.0,6.0,6.0,6.0,6.0,6.0
3,1.0,4.0,6.0,5.0,7.0,5.0,5.0,6.0,6.0,5.0,...,7.0,7.0,7.0,7.0,2.0,7.0,7.0,7.0,7.0,2.0
4,1.0,5.0,6.0,6.0,6.0,5.0,6.0,6.0,5.0,6.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0


In [96]:
numerical_data = numerical_data.astype(float)
numerical_data["Email"] = full_cleaned_data["Email"]
numerical_data = numerical_data.sort_values(["TeamNumber", "TeammateNumber"], ascending=True).reset_index()
full_cleaned_data = full_cleaned_data.sort_values(["TeamNumber", "TeammateNumber"], ascending=True).reset_index()
roster = roster.sort_values(["TeamNumber", "TeammateNumber"], ascending=True).reset_index()

In [97]:
numerical_data

,index,TeamNumber,TeammateNumber,2.1,2.2,2.3,4.1,4.2,4.3,4.4,...,18.2,18.3,18.4,18.5,19.1,19.2,19.3,19.4,19.5,Email
0,0,1.0,1.0,5.0,6.0,5.0,5.0,5.0,6.0,6.0,...,6.0,6.0,6.0,5.0,5.0,6.0,5.0,5.0,4.0,mackenzie.hanh@berkeley.edu
1,1,1.0,2.0,7.0,6.0,7.0,7.0,7.0,6.0,7.0,...,7.0,7.0,7.0,6.0,7.0,7.0,7.0,7.0,7.0,evanhaas@berkeley.edu
2,2,1.0,3.0,5.0,6.0,6.0,6.0,5.0,7.0,6.0,...,6.0,6.0,6.0,2.0,6.0,6.0,6.0,6.0,6.0,jiwondinah.lee@berkeley.edu
3,3,1.0,4.0,6.0,5.0,7.0,5.0,5.0,6.0,6.0,...,7.0,7.0,7.0,2.0,7.0,7.0,7.0,7.0,2.0,mcqueen@berkeley.edu
4,4,1.0,5.0,6.0,6.0,6.0,5.0,6.0,6.0,5.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,mindytsai@berkeley.edu
5,5,2.0,1.0,7.0,7.0,7.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,NaN,7.0,7.0,7.0,7.0,NaN,diarabagayeva@berkeley.edu
6,6,2.0,2.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,7.0,6.0,7.0,NaN,7.0,7.0,6.0,7.0,NaN,adeline_zwl@berkeley.edu
7,7,2.0,3.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,qurratulain.508@berkeley.edu
8,8,2.0,4.0,7.0,7.0,6.0,7.0,7.0,6.0,7.0,...,7.0,7.0,7.0,NaN,7.0,7.0,7.0,7.0,NaN,ivyzhang@berkeley.edu
9,9,3.0,1.0,7.0,7.0,7.0,6.0,6.0,6.0,7.0,...,7.0,7.0,7.0,NaN,7.0,7.0,7.0,7.0,NaN,anamahmed1229@berkeley.edu


In [98]:
#Functions 

def calculate_team_averages(df):
    # Creates a DataFrame with the averages per team for each question
    return df.groupby("TeamNumber").agg(np.nanmean)[numerical_questions]

def get_team_data(team_number):
    # Returns new dataframe containing only TEAM_NUMBER's data
    return numerical_data[numerical_data['TeamNumber'] == team_number]

def get_team_average(team_number, question):
    # Gets TEAM_NUMBER's average on the specified question
    temp_df = get_team_data(team_number)
    return np.nanmean(temp_df[question])

def get_team_std(team_number, teammate_number, question):
    # Gets TEAM_NUMBER's standard deviation on the specified question and should not include self
    temp_df = get_team_data(team_number)
    temp_df = temp_df[temp_df['TeammateNumber'] != teammate_number]
    
    return np.nanstd(temp_df[question])

def get_teammates_average(team_number, teammate_number, question):
    # Gets TEAM_NUMBER's average on the specified question
    # omits data from current student given by TEAMMATE_NUMBER 
    temp_df = get_team_data(team_number)
    
    temp_df = temp_df[temp_df['TeammateNumber'] != teammate_number]
    
    return np.nanmean(temp_df[question])


def get_student_answer(team_number, teammate_number, question):
    # Returns the current student's response for the specified question
    temp_df = get_team_data(team_number)
    
    temp_df = temp_df[temp_df['TeammateNumber'] == teammate_number]
    
    return float(temp_df[question])
    #return temp_df[question]

def get_class_average(question):
    # Returns the class average for QUESTION
    return np.mean(numerical_data[question])

def get_class_stdv(question):
    # Returns the class std dev for QUESTION
    return np.std(numerical_data[question])

## Part 1: Identification Information

In [99]:
roster.head()

,index,FirstName,LastName,Email,TeamNumber,TeammateNumber,Teammate1,Teammate2,Teammate3,Teammate4,Teammate5
0,0,Mackenzie,Cramer,mackenzie.hanh@berkeley.edu,1,1,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
1,1,Evan,Haas,evanhaas@berkeley.edu,1,2,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
2,2,Dinah,Lee,jiwondinah.lee@berkeley.edu,1,3,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
3,3,Catherine,McQueen,mcqueen@berkeley.edu,1,4,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
4,4,Mindy,Tsai,mindytsai@berkeley.edu,1,5,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai


In [100]:
#roster = roster.rename({"FullName": "Full Name"}, axis=1)

In [101]:
#roster["Email"] = roster["Email Address"]

In [102]:
roster["FullName"] = roster["FirstName"] + ' ' + roster["LastName"]

In [104]:
analysis_v2 = pd.DataFrame()

In [106]:
analysis_v2 = roster[["FullName", "Email", "TeamNumber", "TeammateNumber"]]

In [107]:
analysis_v2 = analysis_v2.sort_values(["TeamNumber", "TeammateNumber"], ascending=True).reset_index(drop=True)

In [108]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber
0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1
1,Evan Haas,evanhaas@berkeley.edu,1,2
2,Dinah Lee,jiwondinah.lee@berkeley.edu,1,3
3,Catherine McQueen,mcqueen@berkeley.edu,1,4
4,Mindy Tsai,mindytsai@berkeley.edu,1,5
5,Diara Bagayeva,diarabagayeva@berkeley.edu,2,1
6,Adeline Li,adeline_zwl@berkeley.edu,2,2
7,Sanjida Quarratul Ain,qurratulain.508@berkeley.edu,2,3
8,Ivy Zhang,ivyzhang@berkeley.edu,2,4
9,Anam Ahmed,anamahmed1229@berkeley.edu,3,1


## Part 2: Skills Assessment

In [109]:
...

Ellipsis

## Part 3: Survey Questionnairre

In [110]:
question_shorthand = {}
question_shorthand[2.1] = ['SharedGoalTeamAvg', 'SharedGoalClassAvg', 'SharedGoalClassStDev']
question_shorthand[2.2] = ['IndvGoalTeamAvg', 'IndvGoalClassAvg', 'IndvGoalClassStDev']
question_shorthand[2.3] = ['SupportTeamAvg', 'SupportClassAvg', 'SupportClassStDev']
question_shorthand[3.0] = ['Goal1', 'Goal2', 'Goal3', 'Goal4', 'Goal5']#, 'Goal6', 'Goal7']
#question_shorthand[3.5] = ['NewGoal1', 'NewGoal2', 'NewGoal3', 'NewGoal4', 'NewGoal5', 'NewGoal6', 'NewGoal7']
question_shorthand[4.1] = ['CommunicateTeamAvg', 'CommunicateClassAvg', 'CommunicateClassStDev']
question_shorthand[4.2] = ['WorkAllocTeamAvg', 'WorkAllocClassAvg', 'WorkAllocClassStDev']
question_shorthand[4.3] = ['RoleTeamAvg', 'RoleClassAvg', 'RoleClassStDev']
question_shorthand[4.4] = ['EnjoyTeamAvg', 'EnjoyClassAvg', 'EnjoyClassStDev']
question_shorthand[4.5] = ['ConflictResTeamAvg', 'ConflictResClassAvg', 'ConflictResClassStDev']
question_shorthand[4.6] = ['EfficiencyTeamAvg', 'EfficiencyClassAvg', 'EfficiencyClassStDev']
question_shorthand[4.7] = ['ExpressionTeamAvg', 'ExpressionClassAvg', 'ExpressionClassStDev']

question_shorthand[5.0] = ['MyPoints', 'TeammateAvgPoints', 'ExpectedPoints', 'Percent', 'AveragePointsAllocated', 'StDevPointsAllocated']
question_shorthand[5.1] = 'Work Contribution Teammate 1'
question_shorthand[5.2] = 'Work Contribution Teammate 2'
question_shorthand[5.3] = 'Work Contribution Teammate 3'
question_shorthand[5.4] = 'Work Contribution Teammate 4'
question_shorthand[5.5] = 'Work Contribution Teammate 5'
#question_shorthand[5.6] = 'Work Contribution Teammate 6'
#question_shorthand[5.6] = 'Work Contribution Teammate 7'#might need to increase this based on team size
question_shorthand[6.0] = ['DidWellMe','DidWell1','DidWell2','DidWell3','DidWell4', 'DidWell5']#,'DidWell6','DidWell7']
#question_shorthand[6.5] = ['GPTDidWellMe','GPTDidWell1','GPTDidWell2','GPTDidWell3','GPTDidWell4', 'GPTDidWell5','GPTDidWell6','GPTDidWell7']
question_shorthand[7] = ['DoDiffMe', 'DoDiff1', 'DoDiff2', 'DoDiff3', 'DoDiff4','DoDiff5']#, 'DoDiff6', 'DoDiff7']
#question_shorthand[7.5] = ['GPTDoDiffMe', 'GPTDoDiff1', 'GPTDoDiff2', 'GPTDoDiff3', 'GPTDoDiff4','GPTDoDiff5', 'GPTDoDiff6', 'GPTDoDiff7']
#question_shorthand[9.1] = 'ImproveAmt1'
#question_shorthand[9.2] = 'ImproveAmt2'
#question_shorthand[9.3] = 'ImproveAmt3'
#question_shorthand[10.1] = ["AskFeedbackClassAvg", "AskFeedbackClassStDev"]
#question_shorthand[10.2] = ["AskHelpClassAvg", "AskHelpClassStDev"]
#question_shorthand[10.3] = ["ValueContrClassAvg", "ValueContrClassStdev"]
#question_shorthand[10.4] = ["ReflectClassAvg", "ReflectClassStDev"]
#questions 11-13 are just imported as questions themselves as columns
question_shorthand[14] = ["ContributeMe", "Contribute1", "Contribute2", "Contribute3", "Contribute4", "Contribute5"]# "Contribute6", "Contribute7"]
question_shorthand[15] = ['ImproveMe', 'Improve1', 'Improve2', 'Improve3', 'Improve4', 'Improve5'] #'Improve6', 'Improve7']
question_shorthand[22] = ["GPTContributeMe", "GPTContribute1", "GPTContribute2", "GPTContribute3", "GPTContribute4", "GPTContribute5"]#, "GPTContribute6", "GPTContribute7"]
question_shorthand[23] = ['GPTImproveMe', 'GPTImprove1', 'GPTImprove2', 'GPTImprove3', 'GPTImprove4', 'GPTImprove5']#, 'GPTImprove6', 'GPTImprove7']
question_shorthand[16] = ['InitiativeMe', 'InitiativeTeammateAvg', 'InitiativeClassAvg', 'InitiativeClassStDev']
question_shorthand[17] = ['CommunicateMe', 'CommunicateTeammateAvg', 'Communicate2ClassAvg', 'Communicate2ClassStDev']
question_shorthand[18] = ['ExpertiseMe', 'ExpertiseTeammateAvg', 'ExpertiseClassAvg', 'ExpertiseClassStDev']
question_shorthand[19] = ['RespectMe', 'RespectTeammateAvg', 'RespectClassAvg', 'RespectClassStDev']
#question_shorthand[38.1] = 'ImproveAmt1'
#question_shorthand[38.2] = 'ImproveAmt2'
#question_shorthand[38.3] = 'ImproveAmt3'

### 2.1

In [111]:
#Question 2.1

question_2_1 = dictionary[dictionary["question_id"] == 2.1].reset_index()["question"][0]
analysis_v2[question_2_1] = numerical_data["2.1"]

In [112]:
#Question 2.1 team average
question_2_1_0 = question_shorthand[2.1][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.1"), 1))

analysis_v2[question_2_1_0] = lst_responses

In [113]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg
0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,5.0,5.8
1,Evan Haas,evanhaas@berkeley.edu,1,2,7.0,5.8
2,Dinah Lee,jiwondinah.lee@berkeley.edu,1,3,5.0,5.8
3,Catherine McQueen,mcqueen@berkeley.edu,1,4,6.0,5.8
4,Mindy Tsai,mindytsai@berkeley.edu,1,5,6.0,5.8
5,Diara Bagayeva,diarabagayeva@berkeley.edu,2,1,7.0,7.0
6,Adeline Li,adeline_zwl@berkeley.edu,2,2,7.0,7.0
7,Sanjida Quarratul Ain,qurratulain.508@berkeley.edu,2,3,7.0,7.0
8,Ivy Zhang,ivyzhang@berkeley.edu,2,4,7.0,7.0
9,Anam Ahmed,anamahmed1229@berkeley.edu,3,1,7.0,6.8


In [114]:
#Question 2.1 class avg
question_2_1_1 = question_shorthand[2.1][1]

analysis_v2[question_2_1_1] = round(get_class_average("2.1"), 1)

In [115]:


#Question 2.1 class std
question_2_1_2 = question_shorthand[2.1][2]

analysis_v2[question_2_1_2] = round(get_class_stdv("2.1"), 1)

In [116]:
#analysis_v2.iloc[97]

### 2.2

In [117]:
#Question 2.2

question_2_2 = dictionary[dictionary["question_id"] == 2.2].reset_index()["question"][0]
analysis_v2[question_2_2] = numerical_data["2.2"]

In [118]:
#Question 2.2 team average
question_2_2_0 = question_shorthand[2.2][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.2"), 1))

analysis_v2[question_2_2_0] = lst_responses

In [119]:
#Question 2.2 class avg
question_2_2_1 = question_shorthand[2.2][1]

analysis_v2[question_2_2_1] = round(get_class_average("2.2"), 1)

In [120]:
#Question 2.2 class std
question_2_2_2 = question_shorthand[2.2][2]

analysis_v2[question_2_2_2] = round(get_class_stdv("2.2"), 1)

In [121]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,IndvGoalTeamAvg,IndvGoalClassAvg,IndvGoalClassStDev
0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,5.0,5.8,6.3,0.7,6.0,5.8,6.1,0.9
1,Evan Haas,evanhaas@berkeley.edu,1,2,7.0,5.8,6.3,0.7,6.0,5.8,6.1,0.9
2,Dinah Lee,jiwondinah.lee@berkeley.edu,1,3,5.0,5.8,6.3,0.7,6.0,5.8,6.1,0.9
3,Catherine McQueen,mcqueen@berkeley.edu,1,4,6.0,5.8,6.3,0.7,5.0,5.8,6.1,0.9
4,Mindy Tsai,mindytsai@berkeley.edu,1,5,6.0,5.8,6.3,0.7,6.0,5.8,6.1,0.9
5,Diara Bagayeva,diarabagayeva@berkeley.edu,2,1,7.0,7.0,6.3,0.7,7.0,7.0,6.1,0.9
6,Adeline Li,adeline_zwl@berkeley.edu,2,2,7.0,7.0,6.3,0.7,7.0,7.0,6.1,0.9
7,Sanjida Quarratul Ain,qurratulain.508@berkeley.edu,2,3,7.0,7.0,6.3,0.7,7.0,7.0,6.1,0.9
8,Ivy Zhang,ivyzhang@berkeley.edu,2,4,7.0,7.0,6.3,0.7,7.0,7.0,6.1,0.9
9,Anam Ahmed,anamahmed1229@berkeley.edu,3,1,7.0,6.8,6.3,0.7,7.0,6.2,6.1,0.9


### 2.3

In [122]:
#Question 2.3

question_2_3 = dictionary[dictionary["question_id"] == 2.3].reset_index()["question"][0]
analysis_v2[question_2_3] = numerical_data["2.3"]

In [123]:
#Question 2.3 team average
question_2_3_0 = question_shorthand[2.3][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.3"), 1))

analysis_v2[question_2_3_0] = lst_responses

In [124]:
#Question 2.3 class avg
question_2_3_1 = question_shorthand[2.3][1]

analysis_v2[question_2_3_1] = round(get_class_average("2.3"), 1)

In [125]:
#Question 2.3 class std
question_2_3_2 = question_shorthand[2.3][2]

analysis_v2[question_2_3_2] = round(get_class_stdv("2.3"), 1)

### 3

In [126]:
question_3 = dictionary[dictionary["question_id"] == 3.0].reset_index()["question"][0]

In [127]:
analysis_v2[question_3] = full_cleaned_data["3"]

In [128]:
#goal 1
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 1].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_1 = question_shorthand[3.0][0]
analysis_v2[question_3_1] = lst_responses

In [129]:
#goal 2
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 2].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_2 = question_shorthand[3.0][1]
analysis_v2[question_3_2] = lst_responses

In [130]:
#goal 3
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 3].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_3 = question_shorthand[3.0][2]
analysis_v2[question_3_3] = lst_responses

In [131]:
#goal 4
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 4].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_4 = question_shorthand[3.0][3]
analysis_v2[question_3_4] = lst_responses

In [132]:
#goal 5
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 5].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_5 = question_shorthand[3.0][4]
analysis_v2[question_3_5] = lst_responses

In [133]:
#no adjusted goals

### 4.1

In [134]:
#Question 4.1

question_4_1 = dictionary[dictionary["question_id"] == 4.1].reset_index()["question"][0]
analysis_v2[question_4_1] = numerical_data["4.1"]

In [135]:
#analysis_v2

In [136]:
#Question 4.1 team average
question_4_1_0 = question_shorthand[4.1][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.1"), 1))

analysis_v2[question_4_1_0] = lst_responses

In [137]:
#analysis_v2

In [138]:
#Question 4.1 class avg
question_4_1_1 = question_shorthand[4.1][1]

analysis_v2[question_4_1_1] = round(get_class_average("4.1"), 1)

In [139]:
#Question 4.1 class std
question_4_1_2 = question_shorthand[4.1][2]

analysis_v2[question_4_1_2] = round(get_class_stdv("4.1"), 1)

### 4.2

In [140]:
#Question 4.2

question_4_2 = dictionary[dictionary["question_id"] == 4.2].reset_index()["question"][0]
analysis_v2[question_4_2] = numerical_data["4.2"]

In [141]:
#Question 4.2 team average
question_4_2_0 = question_shorthand[4.2][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.2"), 1))

analysis_v2[question_4_2_0] = lst_responses

In [142]:
#Question 4.2 class avg
question_4_2_1 = question_shorthand[4.2][1]

analysis_v2[question_4_2_1] = round(get_class_average("4.2"), 1)

In [143]:
#Question 4.2 class std
question_4_2_2 = question_shorthand[4.2][2]

analysis_v2[question_4_2_2] = round(get_class_stdv("4.2"), 1)

### 4.3

In [144]:
#Question 4.3

question_4_3 = dictionary[dictionary["question_id"] == 4.3].reset_index()["question"][0]
analysis_v2[question_4_3] = numerical_data["4.3"]

In [145]:
#Question 4.1 team average
question_4_3_0 = question_shorthand[4.3][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.3"), 1))

analysis_v2[question_4_3_0] = lst_responses

In [146]:
#Question 4.3 class avg
question_4_3_1 = question_shorthand[4.3][1]

analysis_v2[question_4_3_1] = round(get_class_average("4.3"), 1)

In [147]:
#Question 4.3 class std
question_4_3_2 = question_shorthand[4.3][2]

analysis_v2[question_4_3_2] = round(get_class_stdv("4.3"), 1)

### 4.4

In [148]:
#Question 4.4

question_4_4 = dictionary[dictionary["question_id"] == 4.4].reset_index()["question"][0]
analysis_v2[question_4_4] = numerical_data["4.4"]

In [149]:
#Question 4.4 team average
question_4_4_0 = question_shorthand[4.4][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.4"), 1))

analysis_v2[question_4_4_0] = lst_responses

In [150]:
#Question 4.4 class avg
question_4_4_1 = question_shorthand[4.4][1]

analysis_v2[question_4_4_1] = round(get_class_average("4.4"), 1)

In [151]:
#Question 4.4 class std
question_4_4_2 = question_shorthand[4.4][2]

analysis_v2[question_4_4_2] = round(get_class_stdv("4.4"), 1)

### 4.5


In [152]:
#Question 4.5

question_4_5 = dictionary[dictionary["question_id"] == 4.5].reset_index()["question"][0]
analysis_v2[question_4_5] = numerical_data["4.5"]

In [153]:
#Question 4.5 team average
question_4_5_0 = question_shorthand[4.5][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.5"), 1))

analysis_v2[question_4_5_0] = lst_responses

In [154]:
#Question 4.5 class avg
question_4_5_1 = question_shorthand[4.5][1]

analysis_v2[question_4_5_1] = round(get_class_average("4.5"), 1)

In [155]:
#Question 4.5 class std
question_4_5_2 = question_shorthand[4.5][2]

analysis_v2[question_4_5_2] = round(get_class_stdv("4.5"), 1)

### 4.6


In [156]:
#Question 4.6

question_4_6 = dictionary[dictionary["question_id"] == 4.6].reset_index()["question"][0]
analysis_v2[question_4_6] = numerical_data["4.6"]

#Question 4.6 team average
question_4_6_0 = question_shorthand[4.6][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.6"), 1))

analysis_v2[question_4_6_0] = lst_responses

#Question 4.6 class avg
question_4_6_1 = question_shorthand[4.6][1]

analysis_v2[question_4_6_1] = round(get_class_average("4.6"), 1)

#Question 4.6 class std
question_4_6_2 = question_shorthand[4.6][2]

analysis_v2[question_4_6_2] = round(get_class_stdv("4.6"), 1)

### 4.7

In [157]:
#Question 4.7

question_4_7 = dictionary[dictionary["question_id"] == 4.7].reset_index()["question"][0]
analysis_v2[question_4_7] = numerical_data["4.7"]

#Question 4.7 team average
question_4_7_0 = question_shorthand[4.7][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.7"), 1))

analysis_v2[question_4_7_0] = lst_responses

#Question 4.7 class avg
question_4_7_1 = question_shorthand[4.7][1]

analysis_v2[question_4_7_1] = round(get_class_average("4.7"), 1)

#Question 4.7 class std
question_4_7_2 = question_shorthand[4.7][2]

analysis_v2[question_4_7_2] = round(get_class_stdv("4.7"), 1)

### 5

In [158]:
#Question 5.1-5.6. will need to change this according to how many people are on a team
question_5_1 = dictionary[dictionary["question_id"] == 5.1].reset_index()["question"][0]
question_5_2 = dictionary[dictionary["question_id"] == 5.2].reset_index()["question"][0]
question_5_3 = dictionary[dictionary["question_id"] == 5.3].reset_index()["question"][0]
question_5_4 = dictionary[dictionary["question_id"] == 5.4].reset_index()["question"][0]
question_5_5 = dictionary[dictionary["question_id"] == 5.5].reset_index()["question"][0]
#question_5_6 = dictionary[dictionary["question_id"] == 5.6].reset_index()["question"][0]
#question_5_7 = dictionary[dictionary["question_id"] == 5.7].reset_index()["question"][0]

analysis_v2[question_5_1] = full_cleaned_data[["5.1"]]
analysis_v2[question_5_2] = full_cleaned_data[["5.2"]]
analysis_v2[question_5_3] = full_cleaned_data[["5.3"]]
analysis_v2[question_5_4] = full_cleaned_data[["5.4"]]
analysis_v2[question_5_5] = full_cleaned_data[["5.5"]]
#analysis_v2[question_5_6] = full_cleaned_data[["5.6"]]
#analysis_v2[question_5_7] = full_cleaned_data[["5.7"]]

In [159]:
#Calculations

#MyPoints
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    student_df = team_df[team_df["TeammateNumber"] == student_teammate_number].reset_index()
    question_needed = '5.' + str(int(student_teammate_number))
    #print(student_df)
    lst_responses.append(student_df[question_needed].reset_index()[question_needed][0])
    
question_5_a = question_shorthand[5][0]
analysis_v2[question_5_a] = lst_responses


In [160]:
#TeammateAvgPoints
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    question_needed = '5.' + str(int(student_teammate_number))
    lst_responses.append(round(get_teammates_average(student_team_number, student_teammate_number, question_needed), 1))

question_5_b = question_shorthand[5][1]
analysis_v2[question_5_b] = lst_responses



In [161]:
#ExpectedPoints
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    lst_responses.append(round(100/team_size, 1))

question_5_c = question_shorthand[5][2]
analysis_v2[question_5_c] = lst_responses

In [162]:
#Percent
question_5_d = question_shorthand[5][3]
analysis_v2[question_5_d] = round((analysis_v2[question_5_b] / analysis_v2[question_5_c]) * 100, 1)

In [163]:
#AveragePointsAllocated....wrong??
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    
    temp_q5_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number][["5.1", "5.2", "5.3", "5.4", "5.5"]]
    temp_q5_df = temp_q5_df.replace({"No Response" : np.nan}).astype(float)
    #sum_points = sum(temp_q5_df.reset_index().drop("index", axis=1).iloc[int(student_teammate_number) - 1])
    #lst_responses.append(round(sum_points/team_size, 1))
    arr = temp_q5_df.reset_index().drop("index", axis=1).dropna().to_numpy().flatten()
    sum_points = np.sum(arr)/team_size
    lst_responses.append(round(sum_points/team_size, 1))

question_5_e = question_shorthand[5][4]
analysis_v2[question_5_e] = lst_responses

In [164]:
#StDevPointsAllocated
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    
    temp_q5_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number][["5.1", "5.2", "5.3", "5.4", "5.5"]]
    temp_q5_df = temp_q5_df.replace({"No Response" : np.nan}).astype(float)
    arr = temp_q5_df.reset_index().drop("index", axis=1).dropna().to_numpy().flatten()
    std = np.std(arr[arr != 0])
    lst_responses.append(round(std, 1))

question_5_f = question_shorthand[5][5]
analysis_v2[question_5_f] = lst_responses

In [165]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,IndvGoalTeamAvg,...,Please allocate 100 points across yourself and your teammates for your contributions to the work of the team. For teammate 2,Please allocate 100 points across yourself and your teammates for your contributions to the work of the team. For teammate 3,Please allocate 100 points across yourself and your teammates for your contributions to the work of the team. For teammate 4,Please allocate 100 points across yourself and your teammates for your contributions to the work of the team. For teammate 5,MyPoints,TeammateAvgPoints,ExpectedPoints,Percent,AveragePointsAllocated,StDevPointsAllocated
0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,5.0,5.8,6.3,0.7,6.0,5.8,...,22,22,22,14,20,21.8,20.0,109.0,20.0,4.0
1,Evan Haas,evanhaas@berkeley.edu,1,2,7.0,5.8,6.3,0.7,6.0,5.8,...,20,21,21,17,20,21.8,20.0,109.0,20.0,4.0
2,Dinah Lee,jiwondinah.lee@berkeley.edu,1,3,5.0,5.8,6.3,0.7,6.0,5.8,...,22,23,23,9,23,21.5,20.0,107.5,20.0,4.0
3,Catherine McQueen,mcqueen@berkeley.edu,1,4,6.0,5.8,6.3,0.7,5.0,5.8,...,23,23,23,8,23,21.5,20.0,107.5,20.0,4.0
4,Mindy Tsai,mindytsai@berkeley.edu,1,5,6.0,5.8,6.3,0.7,6.0,5.8,...,20,20,20,20,20,12.0,20.0,60.0,20.0,4.0
5,Diara Bagayeva,diarabagayeva@berkeley.edu,2,1,7.0,7.0,6.3,0.7,7.0,7.0,...,25,25,25,0,25,24.3,25.0,97.2,25.0,2.7
6,Adeline Li,adeline_zwl@berkeley.edu,2,2,7.0,7.0,6.3,0.7,7.0,7.0,...,25,24,26,0,25,23.3,25.0,93.2,25.0,2.7
7,Sanjida Quarratul Ain,qurratulain.508@berkeley.edu,2,3,7.0,7.0,6.3,0.7,7.0,7.0,...,22,34,22,0,34,24.7,25.0,98.8,25.0,2.7
8,Ivy Zhang,ivyzhang@berkeley.edu,2,4,7.0,7.0,6.3,0.7,7.0,7.0,...,23,25,26,0,26,24.3,25.0,97.2,25.0,2.7
9,Anam Ahmed,anamahmed1229@berkeley.edu,3,1,7.0,6.8,6.3,0.7,7.0,6.2,...,25,25,25,0,25,25.0,25.0,100.0,25.0,1.8


### 6

In [166]:
#Didwellme 

lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    temp_q6_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    response = temp_q6_df[temp_q6_df["TeammateNumber"] == student_teammate_number].reset_index()["6"][0]
    lst_responses.append(response)

question_6_a = question_shorthand[6][0]
analysis_v2[question_6_a] = lst_responses

In [167]:
#DidWell1 thru DidWell6

subset_df = analysis_v2[["TeamNumber", "TeammateNumber", "DidWellMe"]]
#subset_df[subset_df["TeammateNumber"] == 1]
#subset_df

lst_responses_1 = []
lst_responses_2 = []
lst_responses_3 = []
lst_responses_4 = []
lst_responses_5 = []
##lst_responses_6 = []
lst_responses_7 = []


for team in subset_df["TeamNumber"]:#.unique():
    team_series = subset_df[subset_df["TeamNumber"] == team].reset_index()["DidWellMe"]
    if len(team_series) == 1:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(np.nan)
        lst_responses_3.append(np.nan)
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
 #       lst_responses_6.append(np.nan)
 #       lst_responses_7.append(np.nan)
    if len(team_series) == 2:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(np.nan)
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
 #       lst_responses_6.append(np.nan)
 #       lst_responses_7.append(np.nan)
    if len(team_series) == 3:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
  #      lst_responses_6.append(np.nan)
  #      lst_responses_7.append(np.nan)
    if len(team_series) == 4:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(np.nan)
 #       lst_responses_6.append(np.nan)
 #       lst_responses_7.append(np.nan)
    if len(team_series) == 5:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
 #       lst_responses_6.append(np.nan)
 #       lst_responses_7.append(np.nan)
    if len(team_series) == 6:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
 #       lst_responses_6.append(team_series[5])
 #       lst_responses_7.append(np.nan)
    if len(team_series) == 7:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
 #       lst_responses_6.append(team_series[5])
  #      lst_responses_7.append(team_series[6])

question_6_b = question_shorthand[6][1]
question_6_c = question_shorthand[6][2]
question_6_d = question_shorthand[6][3]
question_6_e = question_shorthand[6][4]
question_6_f = question_shorthand[6][5]
#question_6_g = question_shorthand[6][6]
#question_6_h = question_shorthand[6][7]
analysis_v2[question_6_b] = lst_responses_1
analysis_v2[question_6_c] = lst_responses_2
analysis_v2[question_6_d] = lst_responses_3
analysis_v2[question_6_e] = lst_responses_4
analysis_v2[question_6_f] = lst_responses_5
#analysis_v2[question_6_g] = lst_responses_6
#analysis_v2[question_6_h] = lst_responses_7

### 7

In [168]:
#DoDiffMe
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    temp_q7_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    #temp_q7_df = temp_q7_df.replace({"No Response" : np.nan}).astype(float)
    response = temp_q7_df[temp_q7_df["TeammateNumber"] == student_teammate_number].reset_index()["7"][0]
    lst_responses.append(response)

question_7_a = question_shorthand[7][0]
analysis_v2[question_7_a] = lst_responses

In [169]:
#DoDiff1 thru DoDiff6

subset_df = analysis_v2[["TeamNumber", "TeammateNumber", "DoDiffMe"]]
#subset_df[subset_df["TeammateNumber"] == 1]
#subset_df

lst_responses_1 = []
lst_responses_2 = []
lst_responses_3 = []
lst_responses_4 = []
lst_responses_5 = []
#lst_responses_6 = []
#lst_responses_7 = []

for team in subset_df["TeamNumber"]:#.unique():
    team_series = subset_df[subset_df["TeamNumber"] == team].reset_index()["DoDiffMe"]
    if len(team_series) == 1:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(np.nan)
        lst_responses_3.append(np.nan)
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
 #       lst_responses_6.append(np.nan)
 ##       lst_responses_7.append(np.nan)
    if len(team_series) == 2:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(np.nan)
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
  #      lst_responses_6.append(np.nan)
  #      lst_responses_7.append(np.nan)
    if len(team_series) == 3:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
   #     lst_responses_6.append(np.nan)
   #     lst_responses_7.append(np.nan)
    if len(team_series) == 4:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(np.nan)
   #     lst_responses_6.append(np.nan)
   #     lst_responses_7.append(np.nan)
    if len(team_series) == 5:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
   #     lst_responses_6.append(np.nan)
   #     lst_responses_7.append(np.nan)
    if len(team_series) == 6:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
    #    lst_responses_6.append(team_series[5])
    #    lst_responses_7.append(np.nan)
    if len(team_series) == 7:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
    #    lst_responses_6.append(team_series[5])
     #   lst_responses_7.append(team_series[6])
        
question_7_b = question_shorthand[7][1]
question_7_c = question_shorthand[7][2]
question_7_d = question_shorthand[7][3]
question_7_e = question_shorthand[7][4]
question_7_f = question_shorthand[7][5]
#question_7_g = question_shorthand[7][6]
#question_7_h = question_shorthand[7][7]
analysis_v2[question_7_b] = lst_responses_1
analysis_v2[question_7_c] = lst_responses_2
analysis_v2[question_7_d] = lst_responses_3
analysis_v2[question_7_e] = lst_responses_4
analysis_v2[question_7_f] = lst_responses_5
#analysis_v2[question_7_g] = lst_responses_6
#analysis_v2[question_7_h] = lst_responses_7

In [79]:
#GPTDodiff

lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    temp_q7_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    #temp_q7_df = temp_q7_df.replace({"No Response" : np.nan}).astype(float)
    response = temp_q7_df[temp_q7_df["TeammateNumber"] == student_teammate_number].reset_index()["7.5"][0]
    lst_responses.append(response)

question_75_a = question_shorthand[7.5][0]
analysis_v2[question_75_a] = lst_responses

KeyError: '7.5'

In [181]:
#GPT do diff

subset_df = analysis_v2[["TeamNumber", "TeammateNumber", "GPTDoDiffMe"]]
#subset_df[subset_df["TeammateNumber"] == 1]
#subset_df

lst_responses_1 = []
lst_responses_2 = []
lst_responses_3 = []
lst_responses_4 = []
lst_responses_5 = []
lst_responses_6 = []
lst_responses_7 = []

for team in subset_df["TeamNumber"]:#.unique():
    team_series = subset_df[subset_df["TeamNumber"] == team].reset_index()["GPTDoDiffMe"]
    if len(team_series) == 1:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(np.nan)
        lst_responses_3.append(np.nan)
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
        lst_responses_6.append(np.nan)
        lst_responses_7.append(np.nan)
    if len(team_series) == 2:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(np.nan)
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
        lst_responses_6.append(np.nan)
        lst_responses_7.append(np.nan)
    if len(team_series) == 3:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(np.nan)
        lst_responses_5.append(np.nan)
        lst_responses_6.append(np.nan)
        lst_responses_7.append(np.nan)
    if len(team_series) == 4:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(np.nan)
        lst_responses_6.append(np.nan)
        lst_responses_7.append(np.nan)
    if len(team_series) == 5:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
        lst_responses_6.append(np.nan)
        lst_responses_7.append(np.nan)
    if len(team_series) == 6:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
        lst_responses_6.append(team_series[5])
        lst_responses_7.append(np.nan)
    if len(team_series) == 7:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
        lst_responses_6.append(team_series[5])
        lst_responses_7.append(team_series[6])
        
question_75_b = question_shorthand[7.5][1]
question_75_c = question_shorthand[7.5][2]
question_75_d = question_shorthand[7.5][3]
question_75_e = question_shorthand[7.5][4]
question_75_f = question_shorthand[7.5][5]
question_75_g = question_shorthand[7.5][6]
question_75_h = question_shorthand[7.5][7]
analysis_v2[question_75_b] = lst_responses_1
analysis_v2[question_75_c] = lst_responses_2
analysis_v2[question_75_d] = lst_responses_3
analysis_v2[question_75_e] = lst_responses_4
analysis_v2[question_75_f] = lst_responses_5
analysis_v2[question_75_g] = lst_responses_6
analysis_v2[question_75_h] = lst_responses_7

### 9

In [182]:
#analysis_v2[question_shorthand[9.1]] = full_cleaned_data["9.1"]
#analysis_v2[question_shorthand[9.2]] = full_cleaned_data["9.2"]
#analysis_v2[question_shorthand[9.3]] = full_cleaned_data["9.3"]

### 10 (not used here but can use functions below)

In [81]:
#analysis_v2[question_shorthand[10.1][0]] = get_class_average("10.1")
#analysis_v2[question_shorthand[10.1][1]] = get_class_stdv("10.1")

#analysis_v2[question_shorthand[10.2][0]] = get_class_average("10.2")
#analysis_v2[question_shorthand[10.2][1]] = get_class_stdv("10.2")

#analysis_v2[question_shorthand[10.3][0]] = get_class_average("10.3")
#analysis_v2[question_shorthand[10.3][1]] = get_class_stdv("10.3")


### 11

In [82]:
#question_11 = dictionary[dictionary["question_id"] == 11].reset_index()["question"][0]
#analysis_v2[question_11] = full_cleaned_data["11"]

### 12

In [119]:
#question_12 = dictionary[dictionary["question_id"] == 12].reset_index()["question"][0]
#analysis_v2[question_12] = full_cleaned_data["12"]

### 14

In [170]:
#14.1 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '14.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_14_me = question_shorthand[14][0]
analysis_v2[question_14_me] = lst_responses

In [171]:
analysis_v2.head(20)

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,IndvGoalTeamAvg,...,DidWell3,DidWell4,DidWell5,DoDiffMe,DoDiff1,DoDiff2,DoDiff3,DoDiff4,DoDiff5,ContributeMe
0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,5.0,5.8,6.3,0.7,6.0,5.8,...,Our team did well in effectively finishing eac...,We have efficient team meetings and decide act...,I like how after the first feedback where we d...,We seem to frequently miss/pass deadlines we s...,We seem to frequently miss/pass deadlines we s...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,organized/created slides and contributed to sl...
1,Evan Haas,evanhaas@berkeley.edu,1,2,7.0,5.8,6.3,0.7,6.0,5.8,...,Our team did well in effectively finishing eac...,We have efficient team meetings and decide act...,I like how after the first feedback where we d...,Presentation design to make it easier to diges...,We seem to frequently miss/pass deadlines we s...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,wrote main interview usability script and test...
2,Dinah Lee,jiwondinah.lee@berkeley.edu,1,3,5.0,5.8,6.3,0.7,6.0,5.8,...,Our team did well in effectively finishing eac...,We have efficient team meetings and decide act...,I like how after the first feedback where we d...,I think the project needs to be better allocat...,We seem to frequently miss/pass deadlines we s...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,Holistic reviewing of the material and slide r...
3,Catherine McQueen,mcqueen@berkeley.edu,1,4,6.0,5.8,6.3,0.7,5.0,5.8,...,Our team did well in effectively finishing eac...,We have efficient team meetings and decide act...,I like how after the first feedback where we d...,I think that it can be a little frustrating wh...,We seem to frequently miss/pass deadlines we s...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,logs meeting notes and organizes slack messages!
4,Mindy Tsai,mindytsai@berkeley.edu,1,5,6.0,5.8,6.3,0.7,6.0,5.8,...,Our team did well in effectively finishing eac...,We have efficient team meetings and decide act...,I like how after the first feedback where we d...,Sometimes there are tasks that we will all par...,We seem to frequently miss/pass deadlines we s...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,"Ideated using Figma Make, worked on competitiv..."
5,Diara Bagayeva,diarabagayeva@berkeley.edu,2,1,7.0,7.0,6.3,0.7,7.0,7.0,...,I think our team was really friendly and we ha...,"every time when we have different opinions, we...",NaN,to act as a team and be flexible and open to m...,to act as a team and be flexible and open to m...,We should be more respectful of the small dead...,Patience and flexibility.\nWhile we do set goa...,i totally understand that we all take other le...,NaN,"ux research, prototype, slide deck"
6,Adeline Li,adeline_zwl@berkeley.edu,2,2,7.0,7.0,6.3,0.7,7.0,7.0,...,I think our team was really friendly and we ha...,"every time when we have different opinions, we...",NaN,We should be more respectful of the small dead...,to act as a team and be flexible and open to m...,We should be more respectful of the small dead..

In [172]:
#14

question_14_1 = question_shorthand[14][1]
question_14_2 = question_shorthand[14][2]
question_14_3 = question_shorthand[14][3]
question_14_4 = question_shorthand[14][4]
question_14_5 = question_shorthand[14][5]
#question_14_6 = question_shorthand[14][6]
#question_14_7 = question_shorthand[14][7]

feedback_1 = []
feedback_2 = []
feedback_3 = []
feedback_4 = []
feedback_5 = [] 
#feedback_6 = []
#feedback_7 = []#need to adjust to max team size

#possibilities = ["14.1", "14.2", "14.3", "14.4", "14.5", "14.6"]
for i in range(len(full_cleaned_data)):
    my_teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    my_team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_number_str = str(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == my_team_number]
    list_feedback_for_me = list(my_team_df["14." + str(my_teammate_number)])
    num_teammates = len(list_feedback_for_me)
    extra_needed = 7 - num_teammates
    placeholders = [''] * extra_needed
    all_feedback = list_feedback_for_me + placeholders

    feedback_1.append(all_feedback[0])
    feedback_2.append(all_feedback[1])
    feedback_3.append(all_feedback[2])
    feedback_4.append(all_feedback[3])
    feedback_5.append(all_feedback[4])
#    feedback_6.append(all_feedback[5])
#    feedback_7.append(all_feedback[6])
    
analysis_v2[question_14_1] = feedback_1
analysis_v2[question_14_2] = feedback_2
analysis_v2[question_14_3] = feedback_3
analysis_v2[question_14_4] = feedback_4
analysis_v2[question_14_5] = feedback_5
#analysis_v2[question_14_6] = feedback_6
#analysis_v2[question_14_7] = feedback_7

In [173]:
analysis_v2.head()

,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,IndvGoalTeamAvg,...,DoDiff2,DoDiff3,DoDiff4,DoDiff5,ContributeMe,Contribute1,Contribute2,Contribute3,Contribute4,Contribute5
0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,5.0,5.8,6.3,0.7,6.0,5.8,...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,organized/created slides and contributed to sl...,organized/created slides and contributed to sl...,great planner and organizer,"Setting up the materials (ex. google slides, m...",so good at keeping meetings on track and alloc...,"Worked on prototype 2, worked on parts of the ..."
1,Evan Haas,evanhaas@berkeley.edu,1,2,7.0,5.8,6.3,0.7,6.0,5.8,...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,wrote main interview usability script and test...,contributed to slides,wrote main interview usability script and test...,Giving insightful comments on direction,great at volunteering to step up to any task!,Worked on a lot of the insights part of the sl...
2,Dinah Lee,jiwondinah.lee@berkeley.edu,1,3,5.0,5.8,6.3,0.7,6.0,5.8,...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,Holistic reviewing of the material and slide r...,contributed to slides and prototype development,solid figma contributions and presentation org...,Holistic reviewing of the material and slide r...,has great design ideas!,"Did a lot of Figma Make tweaks of prototype 3,..."
3,Catherine McQueen,mcqueen@berkeley.edu,1,4,6.0,5.8,6.3,0.7,5.0,5.8,...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,logs meeting notes and organizes slack messages!,contributed to slides and prototype development,excellent figma prototype and writing down mee...,"Refining the materials, especially related to ...",logs meeting notes and organizes slack messages!,"Did a lot of Figma Make tweaks of prototype 1,..."
4,Mindy Tsai,mindytsai@berkeley.edu,1,5,6.0,5.8,6.3,0.7,6.0,5.8,...,Presentation design to make it easier to diges...,I think the project needs to be better allocat...,I think that it can be a little frustrating wh...,Sometimes there are tasks that we will all par...,"Ideated using Figma Make, worked on competitiv...",contributed to slides,solid competitive analysis,Adding minute details,flexible with tasks,"Ideated using Figma Make, worked on competitiv..."


In [174]:
#22.1 (GPT me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '22.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_25_me = question_shorthand[22][0]
analysis_v2[question_25_me] = lst_responses

In [175]:
#25 (GPT version of 15)

question_22_1 = question_shorthand[22][1]
question_22_2 = question_shorthand[22][2]
question_22_3 = question_shorthand[22][3]
question_22_4 = question_shorthand[22][4]
question_22_5 = question_shorthand[22][5]
#question_25_6 = question_shorthand[25][6]
#question_25_7 = question_shorthand[25][7]

feedback_1 = []
feedback_2 = []
feedback_3 = []
feedback_4 = []
feedback_5 = [] 
#feedback_6 = []
#feedback_7 = []#need to adjust to max team size


for i in range(len(full_cleaned_data)):
    my_teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    my_team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_number_str = str(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == my_team_number]
    list_feedback_for_me = list(my_team_df["22." + str(my_teammate_number)])
    num_teammates = len(list_feedback_for_me)
    extra_needed = 5 - num_teammates
    placeholders = [''] * extra_needed
    all_feedback = list_feedback_for_me + placeholders

    feedback_1.append(all_feedback[0])
    feedback_2.append(all_feedback[1])
    feedback_3.append(all_feedback[2])
    feedback_4.append(all_feedback[3])
    feedback_5.append(all_feedback[4])
#    feedback_6.append(all_feedback[5])
#    feedback_7.append(all_feedback[6])
    
analysis_v2[question_22_1] = feedback_1
analysis_v2[question_22_2] = feedback_2
analysis_v2[question_22_3] = feedback_3
analysis_v2[question_22_4] = feedback_4
analysis_v2[question_22_5] = feedback_5
#analysis_v2[question_25_6] = feedback_6
#analysis_v2[question_25_7] = feedback_7

### 24

### 15

In [176]:
#15.1 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '15.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_15_me = question_shorthand[15][0]
analysis_v2[question_15_me] = lst_responses

In [177]:
#15

question_15_1 = question_shorthand[15][1]
question_15_2 = question_shorthand[15][2]
question_15_3 = question_shorthand[15][3]
question_15_4 = question_shorthand[15][4]
question_15_5 = question_shorthand[15][5]
#question_15_6 = question_shorthand[15][6]
#question_15_7 = question_shorthand[15][7]

feedback_1 = []
feedback_2 = []
feedback_3 = []
feedback_4 = []
feedback_5 = [] 
#feedback_6 = []
#feedback_7 = []#need to adjust to max team size

#possibilities = ["14.1", "14.2", "14.3", "14.4", "14.5", "14.6"]
for i in range(len(full_cleaned_data)):
    my_teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    my_team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_number_str = str(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == my_team_number]
    list_feedback_for_me = list(my_team_df["15." + str(my_teammate_number)])
    num_teammates = len(list_feedback_for_me)
    extra_needed = 7 - num_teammates
    placeholders = [''] * extra_needed
    all_feedback = list_feedback_for_me + placeholders

    feedback_1.append(all_feedback[0])
    feedback_2.append(all_feedback[1])
    feedback_3.append(all_feedback[2])
    feedback_4.append(all_feedback[3])
    feedback_5.append(all_feedback[4])
 #   feedback_6.append(all_feedback[5])
 #   feedback_7.append(all_feedback[6])
    
analysis_v2[question_15_1] = feedback_1
analysis_v2[question_15_2] = feedback_2
analysis_v2[question_15_3] = feedback_3
analysis_v2[question_15_4] = feedback_4
analysis_v2[question_15_5] = feedback_5
#analysis_v2[question_15_6] = feedback_6
#analysis_v2[question_15_7] = feedback_7

### 25

In [178]:
#23.1 (GPT me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '23.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_23_me = question_shorthand[23][0]
analysis_v2[question_23_me] = lst_responses

In [179]:
#23 (GPT version of 15)

question_23_1 = question_shorthand[23][1]
question_23_2 = question_shorthand[23][2]
question_23_3 = question_shorthand[23][3]
question_23_4 = question_shorthand[23][4]
question_23_5 = question_shorthand[23][5]
#question_25_6 = question_shorthand[25][6]
#question_25_7 = question_shorthand[25][7]

feedback_1 = []
feedback_2 = []
feedback_3 = []
feedback_4 = []
feedback_5 = [] 
#feedback_6 = []
#feedback_7 = []#need to adjust to max team size


for i in range(len(full_cleaned_data)):
    my_teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    my_team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_number_str = str(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == my_team_number]
    list_feedback_for_me = list(my_team_df["23." + str(my_teammate_number)])
    num_teammates = len(list_feedback_for_me)
    extra_needed = 5 - num_teammates
    placeholders = [''] * extra_needed
    all_feedback = list_feedback_for_me + placeholders

    feedback_1.append(all_feedback[0])
    feedback_2.append(all_feedback[1])
    feedback_3.append(all_feedback[2])
    feedback_4.append(all_feedback[3])
    feedback_5.append(all_feedback[4])
#    feedback_6.append(all_feedback[5])
#    feedback_7.append(all_feedback[6])
    
analysis_v2[question_23_1] = feedback_1
analysis_v2[question_23_2] = feedback_2
analysis_v2[question_23_3] = feedback_3
analysis_v2[question_23_4] = feedback_4
analysis_v2[question_23_5] = feedback_5
#analysis_v2[question_25_6] = feedback_6
#analysis_v2[question_25_7] = feedback_7

### 16

In [180]:

#16 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '16.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_16_me = question_shorthand[16][0]
analysis_v2[question_16_me] = lst_responses

In [181]:
#Question 16 team average
question_16_tm_avg = question_shorthand[16][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '16.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))
analysis_v2[question_16_tm_avg] = lst_responses

In [182]:
# #Question 16 class avg
question_16_class_avg = question_shorthand[16][2]
class_avg = np.nanmean(full_cleaned_data[["16.1", "16.2", "16.3", "16.4", "16.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_16_class_avg] = round(class_avg, 1)

In [183]:
# #Question 16 class std
question_16_class_std = question_shorthand[16][3]
class_std_arr = full_cleaned_data[["16.1", "16.2", "16.3", "16.4", "16.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_16_class_std] = round(np.std(class_std_arr_no_na), 1)

### 17

In [184]:
# #17 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '17.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_17_me = question_shorthand[17][0]
analysis_v2[question_17_me] = lst_responses

In [185]:
# #Question 17 team average
question_17_tm_avg = question_shorthand[17][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '17.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_17_tm_avg] = lst_responses

In [186]:
#analysis_v2

In [187]:
# #Question 17 class avg
question_17_class_avg = question_shorthand[17][2]
class_avg = np.nanmean(full_cleaned_data[["17.1", "17.2", "17.3", "17.4", "17.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_17_class_avg] = round(class_avg, 1)

In [188]:
# #Question 17 class std
question_17_class_std = question_shorthand[17][3]
class_std_arr = full_cleaned_data[["17.1", "17.2", "17.3", "17.4", "17.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_17_class_std] = round(np.std(class_std_arr_no_na), 1)

### 18

In [189]:
# #18 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '18.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_18_me = question_shorthand[18][0] 
analysis_v2[question_18_me] = lst_responses

In [190]:
# #Question 18 team average
question_18_tm_avg = question_shorthand[18][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '18.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_18_tm_avg] = lst_responses

In [191]:
# #Question 18 class avg
question_18_class_avg = question_shorthand[18][2]
class_avg = np.nanmean(full_cleaned_data[["18.1", "18.2", "18.3", "18.4", "18.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_18_class_avg] = round(class_avg, 1)

In [192]:
# #Question 18 class std
question_18_class_std = question_shorthand[18][3]
class_std_arr = full_cleaned_data[["18.1", "18.2", "18.3", "18.4", "18.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_18_class_std] = round(np.std(class_std_arr_no_na), 1)

### 19

In [193]:
# #19 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '19.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_19_me = question_shorthand[19][0]
analysis_v2[question_19_me] = lst_responses

In [194]:
# #Question 19 team average
question_19_tm_avg = question_shorthand[19][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '19.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_19_tm_avg] = lst_responses

In [195]:
# #Question 19 class avg
question_19_class_avg = question_shorthand[19][2]
class_avg = np.nanmean(full_cleaned_data[["19.1", "19.2", "19.3", "19.4", "19.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_19_class_avg] = round(class_avg, 1)

In [196]:
# #Question 19 class std
question_19_class_std = question_shorthand[19][3]
class_std_arr = full_cleaned_data[["19.1", "19.2", "19.3", "19.4", "19.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_19_class_std] = round(np.std(class_std_arr_no_na), 1)

## Add extra questions

In [197]:
full_cleaned_data.columns

Index(['index', 'Unnamed: 0', 'Email', 'TeamNumber', 'TeammateNumber',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', '2.1',
       '2.2', '2.3', '3', '4.1', '4.5', '4.6', '4.2', '4.3', '4.4', '4.7',
       '5.1', '5.2', '5.3', '5.4', '5.5', '6', '7', '14.1', '14.2', '14.3',
       '14.4', '14.5', '15.1', '15.2', '15.3', '15.4', '15.5', '22.1', '22.2',
       '22.3', '22.4', '22.5', '23.1', '23.2', '23.3', '23.4', '23.5', '16.1',
       '16.2', '16.3', '16.4', '16.5', '17.1', '17.2', '17.3', '17.4', '17.5',
       '18.1', '18.2', '18.3', '18.4', '18.5', '19.1', '19.2', '19.3', '19.4',
       '19.5', 'FirstName', 'LastName'],
      dtype='object')

In [148]:

#analysis_v2[question_shorthand[38.1]] = full_cleaned_data["38.1"]
##analysis_v2[question_shorthand[38.2]] = full_cleaned_data["38.2"]
#analysis_v2[question_shorthand[38.3]] = full_cleaned_data["38.3"]
#analysis_v2["Q25"] = full_cleaned_data['33']
#analysis_v2["What did you learn from the first check-in survey feedback?"] = full_cleaned_data['57']
#analysis_v2["Q33_TEXT"] = full_cleaned_data['33.4.TEXT']
analysis_v2['20.1'] = full_cleaned_data['20.1']
analysis_v2['20.2'] = full_cleaned_data['20.2']
analysis_v2['20.3'] = full_cleaned_data['20.3']
analysis_v2['20.4'] = full_cleaned_data['20.4']
analysis_v2['20.5'] = full_cleaned_data['20.5']
analysis_v2['20.6'] = full_cleaned_data['20.6']
analysis_v2['20.7'] = full_cleaned_data['20.7']
analysis_v2['20.8'] = full_cleaned_data['20.8']
analysis_v2['20.9'] = full_cleaned_data['20.9']
analysis_v2['20.10'] = full_cleaned_data['20.10']
analysis_v2['20.11'] = full_cleaned_data['20.11']
analysis_v2['20.12'] = full_cleaned_data['20.12']
analysis_v2['20.13'] = full_cleaned_data['20.13']
analysis_v2['20.14'] = full_cleaned_data['20.14']
analysis_v2['20.15'] = full_cleaned_data['20.15']
analysis_v2['20.16'] = full_cleaned_data['20.16']
analysis_v2['20.17'] = full_cleaned_data['20.17']
analysis_v2['20.18'] = full_cleaned_data['20.18']
analysis_v2['20.19'] = full_cleaned_data['20.19']
analysis_v2['20.20'] = full_cleaned_data['20.20']
analysis_v2['21.1'] = full_cleaned_data['21.1']
analysis_v2['21.2'] = full_cleaned_data['21.2']
analysis_v2['21.3'] = full_cleaned_data['21.3']
analysis_v2['21.4'] = full_cleaned_data['21.4']
analysis_v2['21.5'] = full_cleaned_data['21.5']
analysis_v2['21.6'] = full_cleaned_data['21.6']
analysis_v2['21.7'] = full_cleaned_data['21.7']
analysis_v2['21.8'] = full_cleaned_data['21.8']
analysis_v2['21.9'] = full_cleaned_data['21.9']
analysis_v2['21.10'] = full_cleaned_data['21.10']
analysis_v2['21.11'] = full_cleaned_data['21.11']
analysis_v2['21.12'] = full_cleaned_data['21.12']
analysis_v2['21.13'] = full_cleaned_data['21.13']
analysis_v2['21.14'] = full_cleaned_data['21.14']
analysis_v2['21.15'] = full_cleaned_data['21.15']
analysis_v2['21.16'] = full_cleaned_data['21.16']
analysis_v2['21.17'] = full_cleaned_data['21.17']
analysis_v2['21.18'] = full_cleaned_data['21.18']
analysis_v2['21.19'] = full_cleaned_data['21.19']
analysis_v2['21.20'] = full_cleaned_data['21.20']
analysis_v2["Q99"] = full_cleaned_data['99']

In [86]:
analysis_v2['25_1']=full_cleaned_data['25.1']
analysis_v2['25_2']=full_cleaned_data['25.2']
analysis_v2['25_3']=full_cleaned_data['25.3']
analysis_v2['26']=full_cleaned_data['26']

In [198]:
analysis_v2['Q25'] = full_cleaned_data['25']

KeyError: '25'

In [199]:
analysis_v2.to_csv("HUTKA_CHECKIN02_MACROANALYSIS_V1.csv")